# Image Classification using cNN

In [1]:
!pip install keras
from keras.models import Sequential # used to initialise NN
from keras.layers import Convolution2D # used to process images which are 2D unlike videos which 3D 
from keras.layers import MaxPooling2D #used for pooling
from keras.layers import Flatten
from keras.layers import Dense

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl


Using TensorFlow backend.


In [2]:
#intialising the CNN
classifier = Sequential()

## PART 1 - Building the CNN

## Step 1. Convolution

In [3]:
# nb_filter is the number of features detector or filters, you have no of row and then no of coloumns 
# border mode - how the borders are handled , not important, default
# input shape - all images dont have the same format , 3 channels - RGB , 256 , 256 pixels - 
# we are using only 64, 64 so it can be faster since we have no gpu# Activation func to obtain non linearity
# Activation func to obtain non linearity
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))


/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3...)`


 ## Step 2. Pooling

In [4]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

## Special step: Another convolutional layer

In [5]:
# we dont need input_shape previous pooled maps would be given automatically
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  from ipykernel import kernelapp as app


 ## Step 3. Flattening

In [6]:
# Flattening all the pooling maps into one same single vector
classifier.add(Flatten())

 ## Step 4. Full Connection

In [7]:
# hidden layer nodes: i/p + o/p/2 , or from experimentation , commmon practice to choose number from power of 2
classifier.add(Dense(output_dim = 128, activation = 'relu'))
# output layer sigmoid- because the output could be cat or dog
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the CNN
# binary outcome so binary_crossentropy
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu")`
  from ipykernel import kernelapp as app
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid")`


In [8]:
#!gsutil -m cp -r  gs://deep-learning-26/notebooks/jupyter/test_set dataset
#!gsutil -m cp -r  gs://deep-learning-26/notebooks/jupyter/training_set dataset


## PART 2 - Fitting the CNN to the images

In [9]:
# image augmentation: 8000 images is not great. More images or a trick. The trick is data augmentation
# augmentation tilts the images changes it and trains in batches
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

classifier.fit_generator(training_set,
                        steps_per_epoch=8000,
                        nb_epoch=25,
                        validation_data=test_set,
                        nb_val_samples=800)



Found 8000 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.


/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:26: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=25, validation_steps=800, steps_per_epoch=8000, validation_data=<keras_pre...)`


Epoch 1/25
8000/8000 [==============================] - 1172s 147ms/step - loss: 0.3364 - acc: 0.8425 - val_loss: 0.7154 - val_acc: 0.8050
Epoch 2/25
8000/8000 [==============================] - 1110s 139ms/step - loss: 0.0771 - acc: 0.9718 - val_loss: 1.0054 - val_acc: 0.7901
Epoch 3/25
8000/8000 [==============================] - 1119s 140ms/step - loss: 0.0430 - acc: 0.9849 - val_loss: 1.1556 - val_acc: 0.7910
Epoch 4/25
8000/8000 [==============================] - 1103s 138ms/step - loss: 0.0314 - acc: 0.9893 - val_loss: 1.3117 - val_acc: 0.7944
Epoch 5/25
8000/8000 [==============================] - 1101s 138ms/step - loss: 0.0255 - acc: 0.9916 - val_loss: 1.4525 - val_acc: 0.7925
Epoch 6/25
8000/8000 [==============================] - 1097s 137ms/step - loss: 0.0217 - acc: 0.9929 - val_loss: 1.3624 - val_acc: 0.8056
Epoch 7/25
8000/8000 [==============================] - 1099s 137ms/step - loss: 0.0203 - acc: 0.9935 - val_loss: 1.4329 - val_acc: 0.8006
Epoch 8/25
8000/8000 [=====

In [31]:
from keras.preprocessing import image
import numpy as np
predict_image = image.load_img('dataset/single_predictions/cat_or_dog_3.jpeg', target_size = (64, 64))
predict_image = image.img_to_array(predict_image)
predict_image = np.expand_dims(predict_image, axis = 0)


In [32]:
y_pred = classifier.predict(predict_image) 
print(y_pred)


[[0.]]


In [33]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [34]:
if y_pred[0][0] == 1:
  prediction = 'dog'
else: 
  prediction = 'cat'

print(prediction)
  

cat
